In [2]:
####Package Imports######
from skimage.metrics import structural_similarity
import imutils
import cv2
import os

####Connecting to Anvil#####
import anvil.server

anvil.server.connect("TATIRUIDUHZPTWDWQSVZR3SN-GAK2P3JOWGIWVQHG")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Authenticated OK


In [7]:
import anvil.media
import numpy as np
import csv


@anvil.server.callable



def compare_image(file1, file2):
    
        with anvil.media.TempFile(file1) as FN1:
            image1 = cv2.imread(FN1)
        with anvil.media.TempFile(file2) as FN2: 
            image2 = cv2.imread(FN2)
            
            
        dim1 = image1.shape
        dim2 = image2.shape
        chk_dim = np.array_equal(dim1,dim2)
        
        ####Normalization###########
        

        
        if (chk_dim is True):
            
            scale_percent = 35

            width1 = int(image1.shape[1] * scale_percent / 100)
            height1 = int(image1.shape[0] * scale_percent / 100)

            width2 = int(image2.shape[1] * scale_percent / 100)
            height2 = int(image2.shape[0] * scale_percent / 100)

            dsize1 = (width1, height1)
            dsize2 = (width2, height2)

            R1 = cv2.resize(image1,dsize1)
            R2 = cv2.resize(image2,dsize2)
        
        
    ####convert to grayscale image###########
    
            grayR1 = cv2.cvtColor(R1, cv2.COLOR_BGR2GRAY)
            grayR2 = cv2.cvtColor(R2, cv2.COLOR_BGR2GRAY)

    ######find Similarity Index###########
       

            (score, diff) = structural_similarity(grayR1, grayR2, full=True)

            diff = (diff * 255).astype("uint8")##convert from floating point to unsigned int for openCV
        
            thresh = cv2.threshold(diff, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
        
            location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
        
            #coords = np.array([])
            
    ##### loop over the contours compute the bounding box draw bounding box on diffrences #####
            
            for c in cnts:
            
                (x, y, w, h) = cv2.boundingRect(c)
                cv2.rectangle(R1, (x, y), (x + w, y + h), (0, 0, 255), 2)
                cv2.rectangle(R2, (x, y), (x + w, y + h), (0, 0, 255), 2)
                coords1 = np.asarray(c)
                coords2 = np.squeeze(coords1, axis = 1)
                
                #coords = np.append(coords, c)
                #print(c)
           
            #print(coords1)
            #print(coords2)
            #print(coords1.shape)
            #print(coords2.shape)
            #fin1 = coords.tolist()
            #np.savetxt((os.path.join(location , 'Coordinates.csv')),coords, delimiter =',')
            #file = os.path.join(location , 'Coordinates.txt')
            
            np.savetxt((os.path.join(location , 'Coordinates.csv')),coords2, fmt = '%.2f', delimiter =',')
            
            #file1 = os.path.join(location , 'Coordinates.csv')
            
            #print(coords)
        #####with open(file, 'w') as f:
            ####for item in fin1:
                #####f.write("%s\t" % item)
        
            #with open(file1, 'w') as f1:
                #wr = csv.writer(f1, quoting=csv.QUOTE_ALL)
                #wr.writerow(fin1)
       
            cv2.imwrite(os.path.join(location , 'Floorplan1.png'), R1)
            cv2.imwrite(os.path.join(location , 'Floorplan2.png'), R2)
            cv2.imwrite(os.path.join(location , 'Difference.png'), diff)
            cv2.imwrite(os.path.join(location , 'Threshold.png'), thresh)
          
        
        #c1 = anvil.media.from_file((os.path.join(location , 'Floorplan1.png')), 'image/png')
        
        #cv2.imshow("Floor Plan 1", R1)
        #cv2.imshow("Floor Plan 2", R2)
        #cv2.imshow("Difference", diff)
        #cv2.imshow("Threshold", thresh)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
            
        #c1 = anvil.media.from_file(R1,'image/png')
        #str1 = ''.join(str(a_list))
        #return(c1)
        
            return(float(score))
        
            
        else:
            
            error = 'PLEASE UPLOAD IMAGES WITH SAME DIMENSIONS'
            return(error)
            

In [4]:
@anvil.server.callable

def return_image():
    location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
    f1 = anvil.media.from_file((os.path.join(location , 'Floorplan1.png')), "img/png")
    f2 = anvil.media.from_file((os.path.join(location , 'Floorplan2.png')), "img/png")
    diff = anvil.media.from_file((os.path.join(location , 'Difference.png')), "img/png")
    th = anvil.media.from_file((os.path.join(location , 'Threshold.png')), "img/png")
        
    return(f1, f2, diff)



In [5]:
@anvil.server.callable

def return_coordinates():
    location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
    coords = anvil.media.from_file((os.path.join(location , 'Coordinates.txt')), "text/plain")
    return coords


In [6]:
@anvil.server.callable

def return_coordinates_csv():
    location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
    coordscsv = anvil.media.from_file((os.path.join(location , 'Coordinates.csv')), "text/csv")
    return coordscsv


In [29]:
@anvil.server.callable

def return_images_download():
    location = 'C:/Users/geral/Documents/SeniorProjectTest/Compare'
    f1_d = anvil.media.from_file((os.path.join(location , 'Floorplan1.png')), "img/png")
    f2_d = anvil.media.from_file((os.path.join(location , 'Floorplan2.png')), "img/png")
    diff_d = anvil.media.from_file((os.path.join(location , 'Difference.png')), "img/png")
    
    return (f1_d,f2_d,diff_d)